In [1]:
# import packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.random_projection import GaussianRandomProjection
from sklearn.base import BaseEstimator
from sklearn.linear_model import Ridge
from sklearn.base import TransformerMixin
%matplotlib inline

In [2]:
names = ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight','Rings']
df = pd.read_csv('abalone.csv',header=None,names=names,sep=';')

In [3]:
df.head()
#df.tail()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [4]:
# We change the M,F and I categorical variables as numerical using 0,1 and 2.
replace_list = {"Sex" : {"M": 0, "F" : 1, "I": 2}}
df.replace(replace_list,inplace=True)

In [5]:
df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,2,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


### predicting abalone dataset with Ridge() & KNeighborsRegressors()

In [6]:
# Split off the feature to predict
targets = df["Rings"]
del df["Rings"]

In [7]:
#df.head()

In [8]:
from sklearn.model_selection import cross_val_score  
# train and test splitting the data
X_train, X_test, y_train, y_test = train_test_split(df, targets, test_size = 0.1, random_state = 0)

In [9]:
# Standrized the data
scaler= StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test = scaler.transform(X_test)

#### Ridge Regression

In [10]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
params={'alpha': [2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
rdg_reg = Ridge()

In [11]:
clf = GridSearchCV(rdg_reg, params, cv=5)
grid_result=clf.fit(df, targets)

In [12]:
print("Best parameters: {}".format(clf.best_params_))

Best parameters: {'alpha': 1.0}


In [13]:
predicted_rings= clf.predict(X_test)

In [14]:
predicted_rings[:5]

array([ 20.48082679,   2.27332733,   1.56235418, -27.98024015,
         3.89358609])

In [15]:
cross_val_score(clf, df, targets, cv=5)

array([0.38285922, 0.21364114, 0.51261091, 0.52738089, 0.45210594])

In [16]:
print("Best cross-validation score: {:.2f}\n".format(clf.best_score_))

Best cross-validation score: 0.42



#### KNeighborsRegressors()

In [17]:
from sklearn.neighbors import KNeighborsRegressor

In [18]:
k = np.arange(30)+1
parameters = {'n_neighbors': k}
k_reg= KNeighborsRegressor()

In [19]:
clf_k = GridSearchCV(k_reg, parameters, cv=5)
result=clf_k.fit(df, targets)

In [20]:
print("Best parameters: {}".format(clf.best_params_))

Best parameters: {'alpha': 1.0}


In [21]:
predicted_rings_k= clf_k.predict(X_test)

In [22]:
predicted_rings_k[:5]

array([ 9.35,  3.8 , 11.45,  3.5 , 12.25])

In [23]:
print("Best cross-validation score: {:.2f}\n".format(clf_k.best_score_))

Best cross-validation score: 0.46



comment: comaring both cross-validation score we can observe that the KNeighborsRegressors has the heighest score.So in this case KNeighborsRegressor is the best estimator.

In [24]:
## Using pipeline  to solve the upper problem:

In [25]:
#parameter for the ridge and knn regression
param_grid = [
        {'regression': [Ridge()], 
         'regression__alpha': [0.001, 0.01, 0.1, 1, 10, 100]},
        {'regression': [KNeighborsRegressor()],
         'regression__n_neighbors': [1,4,5,6,8,10,15,20,100]              
        }
        ]

In [26]:
#create a pipeline
pipe = Pipeline([('preprocessing', StandardScaler()), ('regression',Ridge() )])
#model parameter tuning
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)

#fitting the training data
grid.fit(df, targets)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('preprocessing', StandardScaler(copy=True, with_mean=True, with_std=True)), ('regression', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'regression': [Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)], 'regression__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}, {'regression': [KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=15, p=2,
          weights='uniform')], 'regression__n_neighbors': [1, 4, 5, 6, 8, 10, 15, 20, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
print("Best estimator:\n{}".format(grid.best_estimator_))

Best estimator:
Pipeline(memory=None,
     steps=[('preprocessing', StandardScaler(copy=True, with_mean=True, with_std=True)), ('regression', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=15, p=2,
          weights='uniform'))])


Using pipeline we can also observe that KNeighborsRegressor is the best estimator

## 2. cv_results_

#### cv_results_ for KNeighbours

In [28]:
mean_1=result.cv_results_['mean_test_score']

In [29]:
params_1=result.cv_results_['params']

In [30]:
std=result.cv_results_['std_test_score']

In [31]:
for mean, stdev, param in zip(mean_1, std, params_1):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.060904 (0.139758) with: {'n_neighbors': 1}
0.275194 (0.116394) with: {'n_neighbors': 2}
0.370691 (0.074701) with: {'n_neighbors': 3}
0.407561 (0.073468) with: {'n_neighbors': 4}
0.427059 (0.078014) with: {'n_neighbors': 5}
0.441518 (0.077064) with: {'n_neighbors': 6}
0.444641 (0.078129) with: {'n_neighbors': 7}
0.447876 (0.077244) with: {'n_neighbors': 8}
0.451624 (0.082966) with: {'n_neighbors': 9}
0.454457 (0.080260) with: {'n_neighbors': 10}
0.455084 (0.080908) with: {'n_neighbors': 11}
0.457907 (0.079936) with: {'n_neighbors': 12}
0.456558 (0.080601) with: {'n_neighbors': 13}
0.457239 (0.082925) with: {'n_neighbors': 14}
0.457604 (0.083960) with: {'n_neighbors': 15}
0.455876 (0.085217) with: {'n_neighbors': 16}
0.457041 (0.084799) with: {'n_neighbors': 17}
0.457928 (0.085663) with: {'n_neighbors': 18}
0.457393 (0.086432) with: {'n_neighbors': 19}
0.458627 (0.084267) with: {'n_neighbors': 20}
0.457201 (0.085479) with: {'n_neighbors': 21}
0.456027 (0.087220) with: {'n_neighbors': 2

#### cv_result for Ridge Regression

In [32]:
means=grid_result.cv_results_['mean_test_score']
stdev=grid_result.cv_results_['std_test_score']
params=grid_result.cv_results_['params'] 

In [33]:
for mean, stdev, param in zip(means, stdev, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.420246 (0.110635) with: {'alpha': 2}
0.422431 (0.112687) with: {'alpha': 1.0}
0.422304 (0.113295) with: {'alpha': 0.8}
0.421336 (0.114304) with: {'alpha': 0.5}
0.419769 (0.115005) with: {'alpha': 0.3}
0.418494 (0.115351) with: {'alpha': 0.2}
0.416691 (0.115700) with: {'alpha': 0.1}
0.415510 (0.115887) with: {'alpha': 0.05}
0.414683 (0.116009) with: {'alpha': 0.02}
0.414384 (0.116053) with: {'alpha': 0.01}


### 3. Extreme Learning Machine

In [34]:
names = ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight','Rings']
df_1 = pd.read_csv('abalone.csv',header=None,names=names,sep=';')

In [35]:
replace_list = {"Sex" : {"M": 0, "F" : 1, "I": 2}}
df_1.replace(replace_list,inplace=True)

In [36]:
X =df_1.drop(['Rings'],axis=1)
y = df_1['Rings'].values


In [37]:
#train test split a dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [38]:
class MyModelPart(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_new = X.copy()
        X_new = np.tanh(X_new)
        return X_new

In [44]:
a=np.logspace(-3,3,7)
k=np.linspace(5,1000,7).astype(int)

grid_params1 = [
                  { 'gaussianrandomprojection__n_components':k, 'ridge__alpha':a}
                  ]
pipe_l=make_pipeline(StandardScaler(),
                    GaussianRandomProjection(n_components=25),
                    MyModelPart(),
                    Ridge(),
                    )
pipe_l.named_steps
grid_l = GridSearchCV(pipe_l, param_grid=grid_params1, cv=5, n_jobs=1, verbose=99)



In [45]:
k

array([   5,  170,  336,  502,  668,  834, 1000])

In [46]:
grid_l.fit(X_train, y_train) #I have tried several hours to run this code but it doesn't show any progress.
                             #That's why I can't continue the rest of the part. 

Fitting 5 folds for each of 49 candidates, totalling 245 fits
[CV] gaussianrandomprojection__n_components=5, ridge__alpha=0.001 ....
[CV]  gaussianrandomprojection__n_components=5, ridge__alpha=0.001, score=0.395214654181952, total=   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=5, ridge__alpha=0.001 ....
[CV]  gaussianrandomprojection__n_components=5, ridge__alpha=0.001, score=0.4222863388615055, total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=5, ridge__alpha=0.001 ....
[CV]  gaussianrandomprojection__n_components=5, ridge__alpha=0.001, score=0.46693189114674244, total=   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=5, ridge__alpha=0.001 ....
[CV]  gaussianrandomprojection__n_components=5, ridge__alpha=0.001, score=0.45415785397586

C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (8 < 170).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)
C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (8 < 170).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)
C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (8 < 170).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)
C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of compon

[CV]  gaussianrandomprojection__n_components=170, ridge__alpha=0.001, score=0.5716172751713888, total=   0.0s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    0.3s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=170, ridge__alpha=0.001 ..
[CV]  gaussianrandomprojection__n_components=170, ridge__alpha=0.001, score=0.5507524010961987, total=   0.0s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    0.4s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=170, ridge__alpha=0.001 ..
[CV]  gaussianrandomprojection__n_components=170, ridge__alpha=0.001, score=-0.8865392284404678, total=   0.0s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:    0.4s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=170, ridge__alpha=0.001 ..
[CV]  gaussianrandomprojection__n_components=170, ridge__alpha=0.001, score=0.5310214252761289, total=   0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.5s remaining:    0.0s
[CV] gaussianrandomproject

C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (8 < 170).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)
C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (8 < 170).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)
C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (8 < 170).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)
C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of compon


[CV]  gaussianrandomprojection__n_components=170, ridge__alpha=0.01, score=0.5629328523194541, total=   0.0s
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:    0.6s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=170, ridge__alpha=0.01 ...
[CV]  gaussianrandomprojection__n_components=170, ridge__alpha=0.01, score=0.5552166135248247, total=   0.0s
[Parallel(n_jobs=1)]: Done  43 out of  43 | elapsed:    0.6s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=170, ridge__alpha=0.01 ...
[CV]  gaussianrandomprojection__n_components=170, ridge__alpha=0.01, score=0.5465787758468788, total=   0.0s
[Parallel(n_jobs=1)]: Done  44 out of  44 | elapsed:    0.7s remaining:    0.0s
[CV] gaussianrandomprojection__n_components=170, ridge__alpha=0.01 ...


...

[CV]  gaussianrandomprojection__n_components=1000, ridge__alpha=1000.0, score=0.389833384007757, total=   0.2s
[CV] gaussianrandomprojection__n_components=1000, ridge__alpha=1000.0 


C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (8 < 1000).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)


[CV]  gaussianrandomprojection__n_components=1000, ridge__alpha=1000.0, score=0.40585685228489576, total=   0.2s
[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed:   41.2s finished


C:\Users\Omistaja\Anaconda3\lib\site-packages\sklearn\random_projection.py:378: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (8 < 170).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('gaussianrandomprojection', GaussianRandomProjection(eps=0.1, n_components=25, random_state=None)), ('mymodelpart', MyModelPart()), ('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'gaussianrandomprojection__n_components': array([   5,  170,  336,  502,  668,  834, 1000]), 'ridge__alpha': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=99)

In [47]:
MAE= np.abs(y_test - grid_l.predict(X_test)).mean()

In [49]:
# random forest regressor
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=5)
model.fit(X_train, y_train)
predicted_test_y = model.predict(X_test)
rms_random_forest = np.abs(y_test, predicted_test_y).mean()


In [50]:
rms_random_forest

9.844976076555024

In [51]:
MAE

1.5182909077053766